In [54]:
import sqlite3
import pandas as pd
import plotly.express as px

# Verbinde mit der SQLite-Datenbank
conn = sqlite3.connect('../swiss_travel.db')  # Passe den Pfad ggf. an

# SQL Query
query = """
SELECT h.hotel, h.place_cleaned, h.price_cleaned, p."Ortschaftsname", p.E, p.N
FROM hotels h
INNER JOIN (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY "Ortschaftsname" ORDER BY "Ortschaftsname") AS rn
    FROM places
) p ON h.place_cleaned = p."Ortschaftsname" AND p.rn = 1
"""


df_combined = pd.read_sql_query(query, conn)

In [55]:
df_combined.head()

,hotel,place_cleaned,price_cleaned,Ortschaftsname,E,N
0,Good Night Inn,Brig,138.0,Brig,2643022.054,1129376.217
1,Agriturismo Regusci,Camorino,165.0,Camorino,2721585.340,1112884.534
2,Hotel Drei Könige,Einsiedeln,177.0,Einsiedeln,2698493.177,1221093.950
3,Eringer Hotel,Hérémence,228.0,Hérémence,2596510.782,1108654.451
4,Auberge des Grands Bois,Buchillon,174.0,Buchillon,2520933.442,1146668.837


In [56]:
from pyproj import Transformer

# Create transformer: LV95 to WGS84
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

# Apply transformation
df_combined["lon"], df_combined["lat"] = transformer.transform(df_combined["E"].values, df_combined["N"].values)

df_combined.head()

,hotel,place_cleaned,price_cleaned,Ortschaftsname,E,N,lon,lat
0,Good Night Inn,Brig,138.0,Brig,2643022.054,1129376.217,7.997168,46.314413
1,Agriturismo Regusci,Camorino,165.0,Camorino,2721585.340,1112884.534,9.012619,46.156468
2,Hotel Drei Könige,Einsiedeln,177.0,Einsiedeln,2698493.177,1221093.950,8.736918,47.133499
3,Eringer Hotel,Hérémence,228.0,Hérémence,2596510.782,1108654.451,7.393503,46.129373
4,Auberge des Grands Bois,Buchillon,174.0,Buchillon,2520933.442,1146668.837,6.409271,46.466667


In [57]:
grouped = df_combined.groupby("place_cleaned").agg(
    avg_price=("price_cleaned", "mean"),
    hotel_count=("hotel", "count"),
    lon=("lon", "first"),
    lat=("lat", "first")
).reset_index()

grouped.head()

,place_cleaned,avg_price,hotel_count,lon,lat
0,Aarau,170.666667,3,8.048668,47.389014
1,Adelboden,383.500000,2,7.558185,46.469974
2,Agarn,169.000000,1,7.670872,46.292983
3,Airolo,195.000000,3,8.608554,46.536149
4,Amden,133.000000,1,9.164870,47.159760


In [58]:
fig = px.density_mapbox(grouped,
                        lat="lat",
                        lon="lon",
                        z="hotel_count",  # Use number of hotels for density
                        radius=25,
                        hover_data=["place_cleaned", "avg_price", "hotel_count"],
                        zoom=7,
                        height=800,
                        title="Hotel Density in Switzerland",
                        mapbox_style="carto-positron",
                        color_continuous_scale="Viridis")

fig.show()

/tmp/ipykernel_32756/425338236.py:1: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [59]:
import plotly.express as px

fig = px.density_mapbox(grouped,
                        lat="lat",
                        lon="lon",
                        z="avg_price",
                        radius=25,
                        hover_data=["place_cleaned", "avg_price", "hotel_count"],
                        zoom=7,
                        height=800,
                        title="Heatmap Average Hotel Prices by Location",
                        mapbox_style="carto-positron",
                        color_continuous_scale="Viridis")

fig.show()

/tmp/ipykernel_32756/366412365.py:3: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [60]:
import plotly.express as px

fig = px.scatter_mapbox(grouped,
                        lat="lat",
                        lon="lon",
                        size="hotel_count",
                        color="avg_price",
                        hover_name="place_cleaned",
                        hover_data=["avg_price", "hotel_count"],
                        color_continuous_scale="Plasma",
                        size_max=30,
                        zoom=7,
                        height=800,
                        mapbox_style="carto-positron",
                        title="Combination of both - Swiss Hotels: Bubble Size = Density, Color = Avg Price")

fig.show()

/tmp/ipykernel_32756/2719593717.py:3: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

